# Table of contents

- Decorators
- Context managers
- Iterators
- Generators

Reminder: 15-20 min per topic

# Decorators

A decorator is a function that takes another function and extends its behavior without modifying it directly.

In [ ]:
def logger(func):
    def wrapper():
        print(f"Starting function: {func.__name__}")
        func()
        print(f"Finished function: {func.__name__}")
    return wrapper

def greet():
    print("Welcome to Python!")

@logger
def greet_decorated():
    print("Welcome to Python!")

In [ ]:
greet()

In [ ]:
greet_decorated()

## Closures

A closure is a function that carries some variables from the place it was created, so it can use them later.

The existence of closures follows from the LEGB rule and the ability to treat functions as objects.

In [ ]:
def make_adder(x):
    # adder uses `x` from the enclosing scope (scope: make_adder)
    def adder(y):
        return x + y
    return adder

In [ ]:
add_two  = make_adder(2)
add_five = make_adder(5)

add_two(7) + add_five(10)

In [ ]:
make_adder(2)(7)

<div class="alert alert-danger">
<b>Anti-pattern: </b> using complex closures
</div>

In [ ]:
def value_factory(value=10):
    def get_value():
        return value

    def set_value(new_value):
        nonlocal value
        value = new_value
        return value
    
    return get_value, set_value


get_value, set_value = value_factory(100)
get_value2, set_value2 = value_factory(100)
print(get_value(), get_value2())

set_value(10**6)

print(get_value(), get_value2())


# Decorators

In [ ]:
import sys

def deprecate(func):
    def inner(*args, **kwargs):
        print('{} is deprecated'.format(func.__name__),
              file=sys.stderr)
        return func(*args, **kwargs)
    return inner

def add(x, y):
    return x + y

decorated_add = deprecate(add)

decorated_add(1, 2)

### Decorator syntax

In [ ]:
import sys

def deprecated(func):
    def wrapper(*args, **kwargs):
        print('{} is deprecated'.format(func.__name__),
              file=sys.stderr)
        return func(*args, **kwargs)
    return wrapper

@deprecated
def add(x, y):
    return x + y

add(1, 2)

### Pusheenize

In [ ]:
from IPython import display

def pusheenize(func):
    return display.HTML('<img src="https://media1.tenor.com/images/4a950a1e221d93e654047ecee711af5a/tenor.gif">')

@pusheenize
def dummy_function(arg):
    print(arg)

# same as: dummy_function = pusheenize(dummy_function)

dummy_function

### Problem

Docstrings (and other metadata such as `__name__` or type annotations) are lost when decorating a function.

In [ ]:
deprecated??

In [ ]:
def show(x):
    'This is a really nice looking docstring'
    print(x)

show??

In [ ]:
@deprecated
def show(x):
    'This is a really nice looking docstring'
    print(x)

print(show.__name__)
print(show.__doc__)

In [ ]:
show??

### Solution 1

In [ ]:
def deprecated(func):
    def wrapper(*args, **kwargs):
        print('{} is deprecated!'.format(func.__name__),
              file=sys.stderr)
        return func(*args, **kwargs)
    wrapper.__name__ = func.__name__
    wrapper.__doc__ = func.__doc__
    wrapper.__module__ = func.__module__
    return wrapper

@deprecated
def show(x):
    'This is a really nice looking docstring'
    print(x)

print(show.__name__)
print(show.__doc__)

### Solution 2

Use decorator to solve decorators problem!

In [ ]:
from functools import wraps

def deprecated(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        print('{} is deprecated!'.format(func.__name__),
              file=sys.stderr)
        return func(*args, **kwargs)
    return wrapper

@deprecated
def show(x):
    'This is a really nice looking docstring'
    print(x)

print(show.__name__)
print(show.__doc__)

optional

### Once decorator

In [ ]:
def once(func):
    called = False

    def wrapper(*args, **kwargs):
        nonlocal called
        if not called:
            called = True
            return func(*args, **kwargs)

    return wrapper

@once
def f():
    print('Hi!')
    
@once
def g():
    print("Buy!")


f()
f()

In [ ]:
import functools
import sys

def trace(dest=sys.stderr):
    def wraps(func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            print(
                f'{func.__name__} called with args {args}, {kwargs}!',
                file=dest
            )
            return func(*args, **kwargs)
        return wrapper
    return wraps

@trace(sys.stdout)
def f(x, test):
    if test > 1:
        return f(x, test / 2)

f('Hi!', test=42)

# Context Managers

How can we guarantee that a certain action will run regardless of whether an exception occurred or not?

In [ ]:
def do_something_dangerous(fd):
    raise RuntimeError('Not today!')

fd = open('myfile.txt', 'w')
try:
    do_something_dangerous(fd)
finally:
    print('Closing file')
    fd.close()
    print('File closed')

Context managers provide a convenient way to perform initialization and guaranteed finalization of a “context”.

In [ ]:
r = acquire_resource()
try:
    use_resource(r)
finally:
    release_resource(r)

In [ ]:
with acquire_resource() as r:
    use_resource(r)

Idea: associate `acquire` with `release` so that `release` is called regardless

### Examples

Examples of context managers: `open`

In [ ]:
with open('filename.txt', 'w') as fd:
    fd.write("Hello")
# file is closed
fd.write("world")

Examples of context managers: `tempfile`

In [ ]:
import tempfile

with tempfile.TemporaryFile() as tmp:
    do_something(tmp)
# tmp file is removed

Examples of context managers: `pytest`

In [ ]:
import pytest
with pytest.raises(ZeroDivisionError):
    a = 1 / 0


In [ ]:

# ZeroDivisionError is not expected to occur anymore and will cause test to fail
with pytest.raises(ZeroDivisionError):
    a = 0 / 1

Context managers are objects that implement a special protocol \
In other words, you can create your own context managers by following the protocol.

In [ ]:
import typing as tp
from types import TracebackType

class MyContextManager:
    def __enter__(self) -> tp.Self: # with () as X; analogue of acquire()
        # initialize context
        # e.g. open a file, connect to a database, etc.
        return self
    
    def __exit__(self,
                 exc_type: type[BaseException] | None,
                 exc_value: BaseException | None,
                 traceback: TracebackType | None) -> bool | None:  # analogue of release()
        # finalize context
        # e.g. close a file, disconnect from a database, etc.
        if exc_value is not None:
            return True  # return True from __exit__ to suppress the exception

Semantics

In [ ]:
with acquire_resource() as resource:
    use_resource(resource)

In [ ]:
manager = acquire_resource()  # Manager.__init__
resource = manager.__enter__()
try:
    use_resource(resource)
finally:
    exc_type, exc_value, traceback = sys.exc_info()
    suppress = manager.__exit__(exc_type, exc_value, traceback)
    if exc_value is not None and not suppress:
        raise exc_value

Semi-joking example

In [ ]:
class Tag:
    def __init__(self, name):
        self.name = name
    def __enter__(self):
        print('<{}>'.format(self.name))
        return self
    
    def __exit__(self, *args):
        print('</{}>'.format(self.name))
        
    def update(self, new_name):
        self.name = new_name

        
with Tag('table') as table:
    table.update("broken_table")
    with Tag('tr'):
        with Tag('td'):
            print('cell 1')
        with Tag('td'):
            print('cell 2')

# Iterators

In [ ]:
!touch file.txt

In [ ]:
for i in range(5):
    pass
    
for line in open('file.txt'):
    pass

for key in {'A' : 1, 'B' : 2, 'C' : 3}:
    pass
    
for letter in 'Hello, World':
    pass

In [ ]:
iterable = [1, 2, 3]
iterator = iterable.__iter__()
iterator

In [ ]:
iterator.__next__()

In [ ]:
iterator.__next__()

In [ ]:
iterator.__next__()

In [ ]:
iterator.__next__()

### [Iterable](https://docs.python.org/3/glossary.html#term-iterator)

This is an object that has a defined `__iter__` method, that returns an **iterator**.

Examples: `list`, `dict`, `range`

### [Iterator](https://docs.python.org/3/glossary.html#term-iterator)

This is an object that defines the `__iter__` and `__next__` methods.

The `__iter__` method must return the iterator itself (`self`).

The `__next__` method must return the next element, and if none remain, raise a `StopIteration` exception.

## [iter](https://docs.python.org/3/library/functions.html#iter) & [next](https://docs.python.org/3/library/functions.html#next)

In [ ]:
iterator = iter([1])  # same as [1].__iter__()
iterator

In [ ]:
next(iterator)  # same as iterator.__next__()

In [ ]:
next(iterator)

In [ ]:
next(iterator, 'some value')

## Implementing a `for` loop via `while`

Let's take a deeper look at how `for` loops work.

In [ ]:
for value in sequence:
    ...

In [ ]:
iterator = iter(sequence)
while True:
    try:
        value = next(iterator)
    except StopIteration:
        break
    else:
        ...  # else branch of `for` loop

Iterators are mutable

In [ ]:
iterable = range(10)
iterator = iter(iterable)
same_iterator = iterator
next(iterator), next(iterator), next(iterator)

In [ ]:
next(same_iterator)

### Custom iterators

In [ ]:
class MyRange:
    def __init__(self, start: int, stop: int) -> None:
        self._start = start
        self._stop = stop
        self._index = start

    def __iter__(self) -> 'MyRange':
        return self
    
    def __next__(self) -> int:
        if self._index >= self._stop:
            raise StopIteration()
        value = self._index
        self._index += 1
        return value
    
    
iterable = MyRange(0, 5)
for i in iterable:
    print(i, end=' ')

In [ ]:
iter(iterable)

In [ ]:
iterable is iter(iterable)

Problem: iterators are exhausted after iteration

In [ ]:
iterable = MyRange(0, 5) # iterable and iterator at the same time
for i in iterable:
    print(i, end=' ')


for i in iterable:  # we can't use the same iterator twice
    print(i, end=' ')

Solution: separate iterator and iterable

In [ ]:
class MyRange:
    class Iterator:
        def __init__(self, start: int, stop: int) -> None:
            self._start = start
            self._stop = stop
            self._index = start
            
        def __iter__(self) -> 'MyRange.Iterator':
            return self
        
        def __next__(self) -> int:
            if self._index >= self._stop:
                raise StopIteration()
            value = self._index
            self._index += 1
            return value
    
    def __init__(self, start: int, stop: int) -> None:
        self._start = start
        self._stop = stop

    def __iter__(self) -> Iterator:
        return self.Iterator(self._start, self._stop)
    

In [ ]:
my_range = MyRange(0, 5)

for i in my_range:
    print(i, end=' ')
print()
for i in my_range:
    print(i, end=' ')


### [Sequence](https://docs.python.org/3/glossary.html#term-sequence) as an iterable

In [ ]:
from typing import TypeVar

T = TypeVar('T')

class Sequence:
    def __init__(self, *args: T) -> None:
        self._args = args
        
    def __len__(self) -> int:
        return len(self._args)

    def __getitem__(self, index: int) -> T:
        if index < 0 or index >= len(self):
            raise IndexError(index)  # expected by for to detect eos
        return self._args[index]

In [ ]:
seq = Sequence(1, 2, 3, 4, 5)
seq[0], seq[2], seq[4]

In [ ]:
for i in seq:
    print(i, end=' ')

## [\_\_contains__](https://docs.python.org/3/reference/datamodel.html#object.__contains__)

Sequence has `__contains__` method by default, though iterables don't.

In [ ]:
3 in range(5)

In [ ]:
from typing import Any
# https://docs.python.org/3.11/reference/expressions.html#membership-test-details
# default __contains__ looks like
def __contains__(self, value: Any) -> bool:
    for item in self:
        if item is value or item == value:
            return True
    return False

In [ ]:
class MyRange:
    def __contains__(self, value: int) -> bool:
        return 0 <= value < self._stop
    
    ...

In [ ]:
seq = Sequence(2, 3, 5, 8, 13, 21)

In [ ]:
for i in seq:
    print(i, end=' ')

In [ ]:
8 in seq  # object has no __contains__, so "in" uses iteration over __getitem__

# Generators

A **generator** is a special kind of function that produces values one at a time, instead of returning them all at once.

<details>
<summary>
More formally:
</summary>

A generator is a function that uses `yield` to pause and resume execution, producing a sequence of values over time instead of computing them all at once.
</details>

#### Motivation

* Memory efficiency: You don’t need to keep all results in memory (e.g., numbers from 1 to 1,000,000)
* Lazy evaluation: You only calculate values when asked for, which saves time if you don’t always need everything.
* Async foundations: (later) Generators are the building blocks for `async` and `await` in Python.


In [ ]:
from collections.abc import Iterator

def countdown(n: int) -> Iterator[int]:
    print(f'Counting down from {n}')
    for i in range(n, 0, -1):
        yield i
    print('Done')

In [ ]:
for i in countdown(5):
    print(i)

In [ ]:
countdown

In [ ]:
counter = countdown(10)
counter

In [ ]:
iter(counter) is counter

In [ ]:
counter = countdown(2)

In [ ]:
next(counter)

In [ ]:
next(counter)

In [ ]:
next(counter)

### [Generator](https://docs.python.org/3/glossary.html#term-generator)

This is a special iterator that you get as a result of calling a function that contains the `yield` keyword.

The sequence of values that a generator returns, is defined by the sequence of `yield` statements in the function body.

### Examples

In [ ]:
def squares(size: int) -> Iterator[int]:
    for i in range(size):
        yield i ** 2

In [ ]:
generator = squares(5)

In [ ]:
for elem in generator:
    print(elem, end=' ')

In [ ]:
for elem in generator:
    print(elem, end=' ')

Generators get exhausted!

In [ ]:
from collections.abc import Iterable, Iterator
from typing import TypeVar

T = TypeVar('T')

def unique_ordered(elements: Iterable[T]) -> Iterator[T]:
    seen = set()
    for elem in elements:
        if elem in seen:
            continue
        seen.add(elem)
        yield elem

In [ ]:
for elem in unique_ordered([1, 2, 3, 1, 2, 4]):
    print(elem, end=' ')

### Generator expressions ([generator expression](https://docs.python.org/3/glossary.html#term-generator-expression))

In [ ]:
squares = (x ** 2 for x in range(5))
squares

In [ ]:
for square in squares:
    print(square, end=' ')

In [ ]:
max(x for x in range(10_000_000_000) if x % 11 == 0)

In [ ]:
max([x for x in range(10_000_000_000) if x % 11 == 0])  # ~20G RAM

Where did you take `20G` number from?

In [ ]:
import sys

int_size_bytes = sys.getsizeof(0)
int_count = 10_000_000_000 / 11
list_size_bytes = int_size_bytes * int_count
list_size_gigabytes = list_size_bytes / (1024 ** 3)
list_size_gigabytes

### Generators as iterators

In [ ]:
from collections.abc import Iterator
from dataclasses import dataclass

@dataclass
class BinaryTreeNode:
    value: int
    left: 'BinaryTreeNode | None' = None
    right: 'BinaryTreeNode | None' = None

    def __iter__(self) -> Iterator[int]:  # in-order
        for value in (self.left or ()):
            yield value

        yield self.value

        for value in (self.right or ()):
            yield value

In [ ]:
tree = BinaryTreeNode(
    left=BinaryTreeNode(
        left=BinaryTreeNode(value=1),
        value=2,
    ),
    value=3,
    right=BinaryTreeNode(
        value=4,
        right=BinaryTreeNode(value=5),
    ),
)

In [ ]:
for value in tree:
    print(value, end=' ')